### Loading and saving cleaned dataset in a MySQL DB
#### Steps:
##### - Loading dataset
##### - Testing connection to local server and creating new database
##### - Creating table for Price cars and storing data in it
##### - Running some simple analysis queries through msql connection

#### Importing libraries

In [24]:
#!pip install pymysql

In [26]:
import pandas as pd
# import mysql.connector
# from mysql.connector import Error
import pymysql

In [28]:
df = pd.read_csv('D:/AIjourney/projects/Pride Ads Project/CSV/pride_ads_cleaned_1.csv', encoding='utf-8-sig')
print(f"Dataset shape: {df.shape}")
df.head()

Dataset shape: (539, 10)


,Brand,Name,Model,Trim,Year,Mileage,Fuel,Transmission,Body status,Price
0,pride,صندوق دار,پراید,ساده,1389,355000.0,بنزینی,دنده ای,گلگیر تعویض,345000000
1,pride,صندوق دار,پراید,ساده,1386,325000.0,بنزینی,دنده ای,گلگیر رنگ,320000000
2,pride,151,پراید,GX,1404,0.0,بنزینی,دنده ای,بدون رنگ,680000000
3,pride,131,پراید,SE,1399,40000.0,بنزینی,دنده ای,گلگیر تعویض,560000000
4,pride,131,پراید,SE,1398,88000.0,بنزینی,دنده ای,دو لکه رنگ,600000000


#### Creating connection to MySQL local server

In [30]:
def create_db_connection(host_name, user_name, user_password, db_name=None):
    try:
        connection = pymysql.connect(
            host=host_name,
            user=user_name,
            password=user_password,
            database=db_name,
            charset='utf8mb4',
            cursorclass=pymysql.cursors.Cursor
        )
        print("MySQL Database connection successful")
        return connection
    except Exception as e:  
        print(f"Error: '{e}'")
        return None

def create_database(connection, db_name):
    cursor = connection.cursor()
    try:
        cursor.execute(f"CREATE DATABASE IF NOT EXISTS {db_name} CHARACTER SET utf8mb4 COLLATE utf8mb4_persian_ci")
        print(f"Database '{db_name}' created successfully or already exists")
    except Exception as e:  # Changed from Error to Exception
        print(f"Error creating database: '{e}'")

# ------- MySQL info---------
HOST = "localhost"
USER = "root"  
PASSWORD = ""  # Local db password..
DB_NAME = "iranian_cars_db"

# ++++++++++++ Creating connection and database ++++++++++++++
connection = create_db_connection(HOST, USER, PASSWORD)

create_database(connection, DB_NAME)
# print('Database has been successfully created on MySQL local server')
connection.close()
print('Connection has been closed..')


MySQL Database connection successful
Database 'iranian_cars_db' created successfully or already exists
Connection has been closed..


#### Creating pride_cars table in db

In [33]:
def create_table(connection, db_name):

    cursor = connection.cursor()
    cursor.execute(f"USE {db_name}")
    
    create_table_query = """
    CREATE TABLE IF NOT EXISTS pride_cars (
        id INT AUTO_INCREMENT PRIMARY KEY,
        brand VARCHAR(50),
        name VARCHAR(100),
        model VARCHAR(100),
        trim VARCHAR(50),
        year INT,
        mileage INT,
        fuel VARCHAR(50),
        transmission VARCHAR(50),
        body_status VARCHAR(100),
        price BIGINT,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        INDEX idx_year (year),
        INDEX idx_price (price),
        INDEX idx_trim (trim)
    ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_persian_ci;
    """
    
    try:
        cursor.execute(create_table_query)
        connection.commit()  # Important: commit the changes
        print("Table 'pride_cars' created successfully")
    except Exception as e:
        print(f"Error creating table: '{e}'")
        connection.rollback()

connection = create_db_connection(HOST, USER, PASSWORD, DB_NAME)

if connection:
    create_table(connection, DB_NAME)
else:
    print("Failed to connect to database")

MySQL Database connection successful
Table 'pride_cars' created successfully


#### Inserting cleaned csv dataset into table

In [36]:
def insert_data(connection, df, batch_size=100):
    
    cursor = connection.cursor()

    insert_query = """
    INSERT INTO pride_cars 
    (brand, name, model, trim, year, mileage, fuel, transmission, body_status, price)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    
    # Convert DataFrame to list of tuples
    data_tuples = [
        tuple(x) for x in df[['Brand', 'Name', 'Model', 'Trim', 'Year', 
                              'Mileage', 'Fuel', 'Transmission', 'Body status', 'Price']].values
    ]
    
    total_rows = len(data_tuples)
    inserted_rows = 0
    
    try:
        # Insert in batches for better performance
        for i in range(0, total_rows, batch_size):
            batch = data_tuples[i:i+batch_size]
            cursor.executemany(insert_query, batch)
            connection.commit()
            inserted_rows += len(batch)
            print(f"Inserted batch {i//batch_size + 1}: {len(batch)} rows")
        
        print(f"\nSuccessfully inserted {inserted_rows} out of {total_rows} total rows")
        
    except Exception as e:
        print(f"Error inserting data: '{e}'")
        connection.rollback()
        return False
    
    return True

print(f"Loaded {len(df)} rows from cleaned dataset CSV file..")


success = insert_data(connection, df, batch_size=50)
if success:
    print("Data insertion completed!")
else:
    print("Data insertion failed")

Loaded 539 rows from cleaned dataset CSV file..
Inserted batch 1: 50 rows
Inserted batch 2: 50 rows
Inserted batch 3: 50 rows
Inserted batch 4: 50 rows
Inserted batch 5: 50 rows
Inserted batch 6: 50 rows
Inserted batch 7: 50 rows
Inserted batch 8: 50 rows
Inserted batch 9: 50 rows
Inserted batch 10: 50 rows
Inserted batch 11: 39 rows

✓ Successfully inserted 539 out of 539 total rows
Data insertion completed!


#### Testing some DML queries

In [39]:
def run_test_queries(connection):
    cursor = connection.cursor(pymysql.cursors.DictCursor)  # Return as dictionary
    
    queries = {
        "1. Total rows": "SELECT COUNT(*) as count FROM pride_cars",
        "2. Average price": "SELECT ROUND(AVG(price)) as avg_price FROM pride_cars",
        "3. Price range": """
            SELECT 
                ROUND(MIN(price)) as min_price, 
                ROUND(MAX(price)) as max_price,
                ROUND(AVG(price)) as avg_price
            FROM pride_cars
        """,
        "4. Year distribution": """
            SELECT 
                year, 
                COUNT(*) as count,
                ROUND(AVG(price)) as avg_price
            FROM pride_cars 
            GROUP BY year 
            ORDER BY year DESC
            LIMIT 10
        """,
        "5. Trim distribution": """
            SELECT 
                trim, 
                COUNT(*) as count,
                ROUND(AVG(price)) as avg_price
            FROM pride_cars 
            GROUP BY trim 
            ORDER BY count DESC
        """,
        "6. Sample data": "SELECT * FROM pride_cars LIMIT 3"
    }
    
    print("=" * 60)
    print("DATABASE VERIFICATION RESULTS")
    print("=" * 60)
    
    for name, query in queries.items():
        print(f"\n{name}:")
        print("-" * 40)
        cursor.execute(query)
        results = cursor.fetchall()
        
        if results:
            for row in results:
                # Format output nicely
                formatted_row = []
                for key, value in row.items():
                    if isinstance(value, (int, float)) and key != 'id':
                        # Format large numbers with commas
                        if value > 1000:
                            formatted_value = f"{value:,}"
                        else:
                            formatted_value = str(value)
                    else:
                        formatted_value = str(value)
                    formatted_row.append(f"{key}: {formatted_value}")
                
                print(" | ".join(formatted_row))
        else:
            print("No results")


run_test_queries(connection)

connection.close()                      # IMPORTANTTTTTT ! !!  !

print("\n" + "=" * 60)
print("Connection closed. Database setup complete!")
print("=" * 60)



DATABASE VERIFICATION RESULTS

1. Total rows:
----------------------------------------
count: 539

2. Average price:
----------------------------------------
avg_price: 419796104

3. Price range:
----------------------------------------
min_price: 90,000,000 | max_price: 970,000,000 | avg_price: 419796104

4. Year distribution:
----------------------------------------
year: 1,404 | count: 44 | avg_price: 650247727
year: 1,403 | count: 19 | avg_price: 625421053
year: 1,402 | count: 5 | avg_price: 591000000
year: 1,401 | count: 7 | avg_price: 547571429
year: 1,400 | count: 3 | avg_price: 526000000
year: 1,399 | count: 23 | avg_price: 610043478
year: 1,398 | count: 50 | avg_price: 565660000
year: 1,397 | count: 25 | avg_price: 517080000
year: 1,396 | count: 22 | avg_price: 503454545
year: 1,395 | count: 30 | avg_price: 460100000

5. Trim distribution:
----------------------------------------
trim: SE | count: 239 | avg_price: 523899582
trim: ساده | count: 195 | avg_price: 276893333
trim: 